# Getting started - Requirements
Python (🐍):  This assumes that you have a python environment installed. If you haven't, we recommend installing [Anaconda](https://docs.anaconda.com/anaconda/install/).

FPsim (♀️): This tutorial uses [FPsim v2.0](https://github.com/fpsim/fpsim/tree/rc2.0-methodtime) (still under development). 

To install, you first need to clone or download the a copy of the source code from https://github.com/fpsim/fpsim/
 
```
git clone -b rc2.0-methodtime --single-branch https://github.com/fpsim/fpsim.git
cd fpsim
pip install -e .
```

If it worked, you should be able to import fpsim with `import fpsim as fp`.

<div class="alert alert-info">
    
An interactive version of this notebook is available on [Google Colab](https://colab.research.google.com/github/fpsim/fpsim/blob/main/docs/tutorials/T3_intervention_methods.ipynb?install=fpsim) or [Binder](https://mybinder.org/v2/gh/fpsim/fpsim/HEAD?labpath=docs%2Ftutorials%2FT3_intervention_methods.ipynb).
    
</div>

## Interventions 

At this point, you should now be able to run a single sim, using default or custom parameters and plot the results. 

In this tutorial, you will learn how to use interventions. I

### Basic simulation parameters and method choice module

In [ ]:
import fpsim as fp
import sciris as sc

pars = dict(
    n_agents   = 1_000,
    location   = 'kenya',
    start_year = 2000, 
    end_year   = 2020,
    exposure_factor = 1.0  # Overall scale factor on probability of becoming pregnant
)
method_choice = fp.RandomChoice(location=pars['location'])  # Use RandomChoice for testing.

### Multisim 

It is typical to want to test many interventions and compared with a baseline simulation wihtout the intervention. FPsim has the MultiSim class which handles the running of multiple simulations, }and provides mechanisms to plot the results together, to facilitate comparison. We will use this class here. But first, let's define each of our individual simulations. To distiguish each simulation in the plots, we will pass one more argument to the sim: its 'label'.  

#### 1. A simulation without intervention (baseline)

In [ ]:
s1 = fp.Sim(pars=pars, contraception_module=method_choice, label="Baseline")

#### 2. A simulation with an intervention that changes the EFFICACY of a method

First let's have a look at which contraceptive methods we have available

In [ ]:
Methods = fp.make_methods().Methods
for method in Methods.values(): print(f"{method.idx}: {method.label}")

In [ ]:
change_efficacy_intervention = fp.update_methods(eff={"Injectables": 0.99}, year=2010)  # new efficacy starts in 2010

s2 = fp.Sim(pars=pars, contraception_module=method_choice, 
                 interventions=change_efficacy_intervention, 
                 label="More effective Injectables ")

#### 3. A simulation with an intervention that changes the distribution of DURATIONS on the method


Now, let's try changin a different aspect of the method: the distribution of time-on-method (duration) 

In [ ]:
# The baseline duration for Injectables is a lognormal with parameter par1=2, and par2=3
change_duration_intervention = fp.update_methods(dur_use={'Injectables': dict(dist='lognormal', par1=3, par2=4)}, year=2010) 
s3 = fp.Sim(pars=pars, contraception_module=method_choice, 
                 interventions=change_duration_intervention, 
                 label="Longer time on Injectables")

#### 4. A simulation with an intervention that increases the PROBABILITY of women using contraception (default p_use=0.5)



In [ ]:
change_prob_intervention = fp.update_methods(p_use=0.8, year=2010) 

s4 = fp.Sim(pars=pars, contraception_module=method_choice, 
            interventions=change_prob_intervention, 
            label="Higher probability of contraceptive use")

#### 5. A simulation with an intervention that changes the METHOD MIX (probability of using each available method)


In [ ]:
# The values in method_mix should add up to 1, but if they don't, the intervention update_methods() will take care of that 
change_mix = fp.update_methods(method_mix=[0.25, 0.05, 0.05, 0.0, 0.05, 0.3, 0.1, 0.1, 0.0], year=2010.0)

s5 = fp.Sim(pars=pars, contraception_module=method_choice, 
            interventions=change_mix, 
            label='Different mix')

In [ ]:
simlist = sc.autolist([s1, s2, s3, s4, s5])
msim = fp.MultiSim(sims=simlist)
msim.run(serial=True, compute_stats=False)

In [ ]:
msim.plot(to_plot='cpr');